# 📘 Progress Callbacks for Optimization Monitoring> Monitor optimization progress with progress bars, logging, and early stopping⏱️ **15-20 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Feature Demo**---

## 🎯 Learning ObjectivesAfter this tutorial, you'll be able to:1. Use `ProgressBar` for real-time optimization monitoring2. Log optimization history with `IterationLogger`3. Prevent wasted iterations with `EarlyStopping`4. Combine multiple callbacks using `CallbackChain`5. Create custom callbacks for specialized needs---

## 🔬 Feature Overview**What problem does this solve?**- Long-running fits can appear frozen without feedback- Debugging requires iteration-level details- Optimization may continue unnecessarily after convergence**Available callbacks:**- `ProgressBar`: tqdm-based progress tracking- `IterationLogger`: Detailed logging to file- `EarlyStopping`: Stop when no improvement detected- `CallbackChain`: Combine multiple callbacks- `CallbackBase`: Base class for custom callbacks---

## Setup

In [1]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [2]:
from IPython.display import display

In [3]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

from nlsq import curve_fit
from nlsq.callbacks import (
    CallbackBase,
    CallbackChain,
    EarlyStopping,
    IterationLogger,
    ProgressBar,
)


def exponential_decay(x, amplitude, rate, offset):
    return amplitude * jnp.exp(-rate * x) + offset

# Generate sample data
np.random.seed(42)
x = np.linspace(0, 10, 100)
y_true = 100 * np.exp(-0.5 * x) + 10
y = y_true + np.random.normal(0, 3, size=len(x))

## Example 1: Simple Progress BarMonitor optimization progress with a visual progress bar.

In [4]:
# Create progress bar callback
callback = ProgressBar(max_nfev=50, desc="Fitting exponential")

# Fit with progress bar
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=callback,
    max_nfev=50)

callback.close()

print(f'✓ Fitted: amplitude={popt[0]:.2f}, rate={popt[1]:.3f}, offset={popt[2]:.2f}')

/home/wei/Documents/GitHub/NLSQ/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': 50}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=2.971923e+03 | ‖∇f‖=1.897905e+04 | nfev=1


Fitting exponential:   0%|          | 0/50 [00:00<?, ?it/s]

...

INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=3.487993e+02 | time=1.020s | final_gradient_norm=0.002616448724631404


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 1.416782s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 1.41678211800172, 'final_cost': 697.5985968806078, 'covariance_warning': False}


Fitting exponential:  10%|█         | 5/50 [00:00<00:01, 41.95it/s, cost=3.487993e+02, grad=1.522e-01, iter=4] 

✓ Fitted: amplitude=101.49, rate=0.529, offset=10.45


## Example 2: Iteration LoggingLog detailed optimization progress to a file for later analysis.

In [5]:
# Create logging callback
callback = IterationLogger(
    filename='optimization.log',
    mode='w',
    log_params=True  # Include parameter values
)

# Fit with logging
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=callback,
    max_nfev=50)

callback.close()

print('✓ Log written to optimization.log')
print('First few lines:')
with open('optimization.log') as f:
    print(''.join(f.readlines()[:10]))

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': 50}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=2.971923e+03 | ‖∇f‖=1.897905e+04 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=3.775535e+02 | ‖∇f‖=7.532724e+01 | step=8.015710e+01 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.273605s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=3.487993e+02 | time=0.274s | final_gradient_norm=0.002616448724631404


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.479344s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.47934373700991273, 'final_cost': 697.5985968806078, 'covariance_warning': False}


✓ Log written to optimization.log
First few lines:
NLSQ Optimization Log
Started: 2025-12-18 00:39:06

Iter    1 | Cost: 3.775535e+02 | Grad: 1.898e+04 | NFev:    2 | Time: 0.00s | Params: [99.345633,  0.541293, 11.578664]
Iter    2 | Cost: 3.488163e+02 | Grad: 7.533e+01 | NFev:    3 | Time: 0.01s | Params: [101.488582,   0.528776,  10.460917]
Iter    3 | Cost: 3.487993e+02 | Grad: 7.138e+01 | NFev:    4 | Time: 0.01s | Params: [101.493062,   0.528967,  10.448703]
Iter    4 | Cost: 3.487993e+02 | Grad: 1.522e-01 | NFev:    5 | Time: 0.02s | Params: [101.493192,   0.52897 ,  10.448796]



## Example 3: Early StoppingPrevent wasted iterations by stopping when optimization stalls.

In [6]:
# Create early stopping callback
callback = EarlyStopping(
    patience=10,       # Stop after 10 iterations without improvement
    min_delta=1e-6,    # Minimum improvement threshold
    verbose=True)

# Fit with early stopping
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=callback,
    max_nfev=1000  # Set high, early stopping prevents waste
)

print(f'✓ Fitted: amplitude={popt[0]:.2f}, rate={popt[1]:.3f}, offset={popt[2]:.2f}')

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': 1000}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=2.971923e+03 | ‖∇f‖=1.897905e+04 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=3.775535e+02 | ‖∇f‖=7.532724e+01 | step=8.015710e+01 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.272001s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=3.487993e+02 | time=0.272s | final_gradient_norm=0.002616448724631404


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.480781s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.4807813279912807, 'final_cost': 697.5985968806078, 'covariance_warning': False}


✓ Fitted: amplitude=101.49, rate=0.529, offset=10.45


## Example 4: Combining Multiple CallbacksUse `CallbackChain` to combine progress bar, logging, and early stopping.

In [7]:
# Combine multiple callbacks
callback = CallbackChain(
    ProgressBar(max_nfev=50, desc="Optimizing"),
    IterationLogger('combined.log', log_params=False),
    EarlyStopping(patience=10, verbose=False))

# Fit with callback chain
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=callback,
    max_nfev=50)

callback.close()

print('✓ All callbacks executed!')
print('Check combined.log for iteration history')

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': 50}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=2.971923e+03 | ‖∇f‖=1.897905e+04 | nfev=1


Optimizing:   0%|          | 0/50 [00:00<?, ?it/s]

...

INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=3.487993e+02 | time=0.274s | final_gradient_norm=0.002616448724631404


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.496498s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.4964978629723191, 'final_cost': 697.5985968806078, 'covariance_warning': False}


Optimizing:  10%|█         | 5/50 [00:00<00:00, 50.33it/s, cost=3.487993e+02, grad=1.522e-01, iter=4] 

✓ All callbacks executed!
Check combined.log for iteration history


## Example 5: Custom CallbackCreate specialized callbacks by subclassing `CallbackBase`.

In [8]:
class BestParameterTracker(CallbackBase):
    """Custom callback to track best parameters."""

    def __init__(self):
        self.best_cost = np.inf
        self.best_params = None
        self.history = []

    def __call__(self, iteration, cost, params, info):
        """Track best parameters."""
        self.history.append({'iter': iteration, 'cost': cost})

        if cost < self.best_cost:
            self.best_cost = cost
            self.best_params = params.copy()
            print(f'  → New best at iter {iteration}: cost={cost:.6f}')

    def get_best(self):
        return self.best_params, self.best_cost

# Use custom callback
tracker = BestParameterTracker()
popt, pcov = curve_fit(
    exponential_decay, x, y,
    p0=[80, 0.4, 5],
    callback=tracker,
    max_nfev=50)

best_params, best_cost = tracker.get_best()
print(f'\n✓ Best cost: {best_cost:.6f}')
print(f'✓ Final params match best: {np.allclose(popt, best_params)}')

INFO:nlsq.curve_fit:Starting curve fit | {'n_params': 3, 'n_data_points': 100, 'method': 'trf', 'solver': 'auto', 'batch_size': None, 'has_bounds': False, 'dynamic_sizing': False}


INFO:nlsq.least_squares:Starting least squares optimization | {'method': 'trf', 'n_params': 3, 'loss': 'linear', 'ftol': 1e-08, 'xtol': 1e-08, 'gtol': 1e-08}


INFO:nlsq.optimizer.trf:Starting TRF optimization (no bounds) | {'n_params': 3, 'n_residuals': 100, 'max_nfev': 50}


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=0 | cost=2.971923e+03 | ‖∇f‖=1.897905e+04 | nfev=1


PERFORMANCE:nlsq.optimizer.trf:Optimization: iter=1 | cost=3.775535e+02 | ‖∇f‖=7.532724e+01 | step=8.015710e+01 | nfev=2


...

PERFORMANCE:nlsq.least_squares:Timer: optimization took 0.262403s


INFO:nlsq.least_squares:Convergence: reason=`ftol` termination condition is satisfied. | iterations=4 | final_cost=3.487993e+02 | time=0.262s | final_gradient_norm=0.002616448724631404


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit took 0.489592s


INFO:nlsq.curve_fit:Curve fit completed | {'total_time': 0.489592302008532, 'final_cost': 697.5985968806078, 'covariance_warning': False}


  → New best at iter 1: cost=377.553481
  → New best at iter 2: cost=348.816317
  → New best at iter 3: cost=348.799299
  → New best at iter 4: cost=348.799298

✓ Best cost: 348.799298
✓ Final params match best: True


## Visualize Results

In [9]:
y_fit = exponential_decay(x, *popt)

fig = plt.figure(figsize=(10, 4))

plt.subplot(121)
plt.plot(x, y, 'o', alpha=0.3, label='Data')
plt.plot(x, y_true, 'g--', label='True')
plt.plot(x, y_fit, 'r-', linewidth=2, label='Fitted')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('Exponential Decay Fit')

plt.subplot(122)
plt.plot(x, y - y_fit, '.')
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('x')
plt.ylabel('Residuals')
plt.title('Fit Quality')

plt.tight_layout()
plt.tight_layout()
plt.show()


/tmp/ipykernel_745354/2073915633.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 💡 Key Insights1. **Progress bars** provide real-time feedback for long-running fits2. **Iteration logging** enables detailed post-analysis and debugging3. **Early stopping** saves computation when optimization stalls4. **Callback chains** combine multiple monitoring strategies5. **Custom callbacks** enable specialized monitoring for your use case---## 📚 Best Practices- Use `ProgressBar` for interactive work (notebooks, scripts)- Use `IterationLogger` for production/automated workflows- Set `patience` in `EarlyStopping` based on problem complexity- Combine callbacks with `CallbackChain` for comprehensive monitoring- Extend `CallbackBase` for custom visualization or metrics---## 🎓 Next Steps- Try callbacks on your own optimization problems- Create custom callbacks for domain-specific metrics- Explore callback integration with logging frameworks- Use callbacks to implement adaptive optimization strategies---